In [5]:
%matplotlib

Using matplotlib backend: MacOSX


In [10]:
from pathlib import Path
import re
from pprint import pprint

import numpy as np
from scipy import signal, ndimage
import matplotlib.pyplot as plt
from matplotlib import pyplot

import emd
import eelbrain
import mne
import trftools

## How to get the IF from the envelope, and turn it into the TRF?

1. Load in the envelope
2. Emd sift the envelope
3. Save the IF value
4. Pack the IF value with the sensor
5. Calculate the IF's TRF

In [11]:
## NATIVES ##

STIMULI = [str(i) for i in range(1, 13)]
DATA_ROOT = Path("/Volumes/Neurolang_1/Master Program/New_Thesis_topic/Experiments_Results")  #Path("~").expanduser() / 'Data' / 'Alice'
PREDICTOR_audio_DIR = DATA_ROOT / 'TRFs_pridictors/audio_predictors'
PREDICTOR_word_DIR = DATA_ROOT / 'TRFs_pridictors/word_predictors'
EEG_DIR = DATA_ROOT / 'EEG_Natives' / 'Alice_natives_ICAed_fif'
Native_SUBJECTS = [path.name for path in EEG_DIR.iterdir() if re.match(r'S\d*', path.name)]
# Define a target directory for TRF estimates and make sure the directory is created
TRF_DIR = DATA_ROOT / 'TRFs_Natives'
TRF_DIR.mkdir(exist_ok=True)
print(Native_SUBJECTS)
print(len(Native_SUBJECTS))

['S13_Alice-natives_sfreq-100_raw.fif', 'S14_Alice-natives_sfreq-100_raw.fif', 'S12_Alice-natives_sfreq-100_raw.fif', 'S15_Alice-natives_sfreq-100_raw.fif', 'S11_Alice-natives_sfreq-100_raw.fif', 'S16_Alice-natives_sfreq-100_raw.fif', 'S17_Alice-natives_sfreq-100_raw.fif', 'S18_Alice-natives_sfreq-100_raw.fif', 'S19_Alice-natives_sfreq-100_raw.fif', 'S20_Alice-natives_sfreq-100_raw.fif', 'S21_Alice-natives_sfreq-100_raw.fif', 'S01_Alice-natives_sfreq-100_raw.fif', 'S03_Alice-natives_sfreq-100_raw.fif', 'S04_Alice-natives_sfreq-100_raw.fif', 'S05_Alice-natives_sfreq-100_raw.fif', 'S06_Alice-natives_sfreq-100_raw.fif', 'S08_Alice-natives_sfreq-100_raw.fif', 'S10_Alice-natives_sfreq-100_raw.fif', 'S22_Alice-natives_sfreq-100_raw.fif', 'S25_Alice-natives_sfreq-100_raw.fif', 'S26_Alice-natives_sfreq-100_raw.fif', 'S34_Alice-natives_sfreq-100_raw.fif', 'S35_Alice-natives_sfreq-100_raw.fif', 'S36_Alice-natives_sfreq-100_raw.fif', 'S37_Alice-natives_sfreq-100_raw.fif', 'S38_Alice-natives_sfreq

In [45]:
# Load the broad-band envelope and process it in the same way
envelope = [eelbrain.load.unpickle(PREDICTOR_audio_DIR / f'{stimulus}~gammatone-1.pickle') for stimulus in STIMULI]  # Load in the data
"""# Code Explanation
envelopeL = []
for stimulus in STIMULI:
    envelope = eelbrain.load.unpickle(PREDICTOR_audio_DIR / f'{stimulus}~gammatone-1.pickle')
    print(envelope)
    envelopeL.append(envelope)
"""

envelope = [x.bin(0.01, dim='time', label='start') for x in envelope]
print(envelope)
""" # Code Explanation
envelopeL_2 = []
for x in envelope:
    envelope = x.bin(0.01, dim='time', label='start')
    envelopeL_2.append(envelope)
"""
envelope = [trftools.pad(x, tstart=-0.100, tstop=x.time.tstop + 1, name='envelope') for x in envelope]
print(envelope)

# Try
env_x_AllLIST = []
for env_x in envelope:
    env_xLIST = []
    for i in range(len(env_x)):
        x = env_x[i]
        print(x)
    print()
    print()
        #env_xLIST.append(x)
    #env_x_AllLIST.append(env_xLIST)
#print(env_x_AllLIST)


[<NDVar 'DownTheRabbitHoleFinal_SoundFile1.wav': 5753 time>, <NDVar 'DownTheRabbitHoleFinal_SoundFile2.wav': 6084 time>, <NDVar 'DownTheRabbitHoleFinal_SoundFile3.wav': 6325 time>, <NDVar 'DownTheRabbitHoleFinal_SoundFile4.wav': 6998 time>, <NDVar 'DownTheRabbitHoleFinal_SoundFile5.wav': 6627 time>, <NDVar 'DownTheRabbitHoleFinal_SoundFile6.wav': 6377 time>, <NDVar 'DownTheRabbitHoleFinal_SoundFile7.wav': 6289 time>, <NDVar 'DownTheRabbitHoleFinal_SoundFile8.wav': 5730 time>, <NDVar 'DownTheRabbitHoleFinal_SoundFile9.wav': 5722 time>, <NDVar 'DownTheRabbitHoleFinal_SoundFile10.wav': 6126 time>, <NDVar 'DownTheRabbitHoleFinal_SoundFile11.wav': 5616 time>, <NDVar 'DownTheRabbitHoleFinal_SoundFile12.wav': 4698 time>]
[<NDVar 'envelope': 5862 time>, <NDVar 'envelope': 6193 time>, <NDVar 'envelope': 6434 time>, <NDVar 'envelope': 7107 time>, <NDVar 'envelope': 6736 time>, <NDVar 'envelope': 6487 time>, <NDVar 'envelope': 6398 time>, <NDVar 'envelope': 5839 time>, <NDVar 'envelope': 5831 tim

IndexError: index 5910 is out of bounds for axis 0 with size 5862

In [41]:
plt.plot(envelope.x)

AttributeError: 'list' object has no attribute 'x'

In [7]:
sample_rate = 1000
seconds = 10
num_samples = sample_rate*seconds

time_vect = np.linspace(0, seconds, num_samples)

freq = 5

# Change extent of deformation from sinusoidal shape [-1 to 1]
nonlinearity_deg = 0.25

# Change left-right skew of deformation [-pi to pi]
nonlinearity_phi = -np.pi/4

# Compute the signal

# Create a non-linear oscillation
x = emd.simulate.abreu2010(freq, nonlinearity_deg, nonlinearity_phi, sample_rate, seconds)

x += np.cos(2 * np.pi * 1 * time_vect)        # Add a simple 1Hz sinusoid
x -= np.sin(2 * np.pi * 2.2e-1 * time_vect)   # Add part of a very slow cycle as a trend

# Visualise the time-series for analysis
plt.figure(figsize=(12, 4))
plt.plot(x)

In [8]:
# Define and simulate a simple signal
peak_freq = 15
sample_rate = 256
seconds = 10
noise_std = .4
x = emd.simulate.ar_oscillator(peak_freq, sample_rate, seconds,
                               noise_std=noise_std, random_seed=42, r=.96)[:, 0]
x = x*1e-4
t = np.linspace(0, seconds, seconds*sample_rate)

# sphinx_gallery_thumbnail_number = 6


# Plot the first 5 seconds of data
plt.figure(figsize=(10, 2))
plt.plot(t[:sample_rate*3], x[:sample_rate*3], 'k')